# Preprocess Input Data and Create processed Json file

---

---

# 1. Set up

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
# module_path = "../../../utils"
# sys.path.append(os.path.abspath(module_path))
# print(os.path.abspath(module_path))

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))
print(os.path.abspath(module_path))

/root/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs/utils


# 2. Remvoe duplicated contents 

In [3]:
from utils.proc_docs import get_load_json, show_doc_json

In [4]:
import glob
import pandas as pd

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/*.json'
json_files = glob.glob(folder_path)

def remove_duplicate_col(json_files, remove_col):
    '''
    json_files : list of json file path
    remove_col : column name to remove duplicate
    Desc: 
    '''
    all_df = pd.DataFrame()
    
    for file_path in json_files:
        df = pd.read_json(file_path)
        df = pd.json_normalize(df.sections) # Normalize sections
        all_df = pd.concat([all_df, df])
    
    all_df.drop_duplicates(subset=remove_col, inplace=True)
    
    return all_df

df = remove_duplicate_col(json_files, remove_col='content')
df

,title,last_updated,content,url,project
0,AR demo barcodes,2023-10-16,AR demo barcodes. Use the following barcodes t...,https://docs.samsungknox.com/admin/knox-captur...,KCAP
1,Videos,2023-07-26,Videos. This section contains product and how-...,https://docs.samsungknox.com/admin/knox-captur...,KCAP
2,Overview,2023-07-26,Overview. Get started with the basics by learn...,https://docs.samsungknox.com/admin/knox-captur...,KCAP
3,Troubleshoot,2023-07-26,Troubleshoot. Access support from the app To a...,https://docs.samsungknox.com/admin/knox-captur...,KCAP
4,Knox Capture,2023-10-16,Knox Capture. Knox Capture is an Android solut...,https://docs.samsungknox.com/admin/knox-capture,KCAP
...,...,...,...,...,...
129,Which types of companies are best suited to us...,2023-07-26,Which types of companies are best suited to us...,https://docs.samsungknox.com/admin/knox-config...,KC
130,Will end users be able to unlink Samsung devic...,2023-07-26,Will end users be able to unlink Samsung devic...,https://docs.samsungknox.com/admin/knox-config...,KC
131,Does Samsung customize booting/shutdown animat...,2023-07-26,Does Samsung customize booting/shutdown animat...,https://docs.samsungknox.com/admin/knox-config...,KC
132,Can I get support from Samsung for any issues ...,2023-07-26,Can I get support from Samsung for any issues ...,https://docs.samsungknox.com/admin/knox-config...,KC


# 3. Craete new json file

In [5]:
cleaned_json_file_path = "data/poc/preprocessed_json/all_processed_data.json"
df.reset_index(drop=True).to_json(cleaned_json_file_path,orient='records')

Verify the created json file

In [6]:
df_json = pd.read_json(cleaned_json_file_path)
df_json.head(3)

,title,last_updated,content,url,project
0,AR demo barcodes,2023-10-16,AR demo barcodes. Use the following barcodes t...,https://docs.samsungknox.com/admin/knox-captur...,KCAP
1,Videos,2023-07-26,Videos. This section contains product and how-...,https://docs.samsungknox.com/admin/knox-captur...,KCAP
2,Overview,2023-07-26,Overview. Get started with the basics by learn...,https://docs.samsungknox.com/admin/knox-captur...,KCAP


In [7]:
%store cleaned_json_file_path

Stored 'cleaned_json_file_path' (str)


# 4. Translate English to Korean

In [8]:
import boto3

translate_client = boto3.client(service_name='translate', region_name='us-east-1', use_ssl=True)



## Text Splitter 정의 
- AWS Translate 는 최대 10000 bytes 를 번역할 수 있어서, 조금 더 작은 단위로 문서를 쪼개서 번역 함

In [10]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter

chunk_size = 8192
chunk_overlap = 0


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)




In [11]:
%%time

import pandas as pd

def translate_df(df, target_col, new_col):
    
    def run_translate(text):
        '''
        Split doc into small chunks, translate them, and then merge into one
        '''
        doc = [Document(page_content=text)]
        docs = text_splitter.split_documents(doc)                
        
        translate_doc = []
        for i, chunk in enumerate(docs):
            # print("English Text: \n", chunk.page_content)                    
            result = translate_client.translate_text(Text=chunk.page_content, 
                    SourceLanguageCode="en", TargetLanguageCode="ko")
            result = result['TranslatedText']
            translate_doc.append(result)
        # print("# of chunks: ", i)
            
        translate_doc = ' '.join(translate_doc)            
        # print("translate_doc: \n", translate_doc)
        
        return translate_doc
    
    def translate(df):
        text = df[target_col]
        result = run_translate(text)
        # print(result)        

        return result
        
    # Start here
    df[new_col] = df.apply(translate, axis=1)
    
    return df




CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 11.7 µs


## 번역 실행

In [13]:
%%time
df_kr = translate_df(df, target_col='title', new_col='title_ko')
df_kr = translate_df(df_kr, target_col='content', new_col='content_ko')


CPU times: user 5.1 s, sys: 158 ms, total: 5.26 s
Wall time: 1min 38s


In [14]:
def post_process_translate(df):
    df.drop(columns=['title', 'content'], inplace=True)
    df.rename(columns={'title_ko': 'title', 'content_ko': 'content'}, inplace=True)

    return df

df_post = post_process_translate(df_kr)
df_post

,last_updated,url,project,title,content
0,2023-10-16,https://docs.samsungknox.com/admin/knox-captur...,KCAP,AR 데모 바코드,AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드...
1,2023-07-26,https://docs.samsungknox.com/admin/knox-captur...,KCAP,동영상,동영상.이 섹션에는 Knox Capture와 관련된 제품 및 사용 방법 비디오가 포...
2,2023-07-26,https://docs.samsungknox.com/admin/knox-captur...,KCAP,개요,개요.Knox Capture가 무엇이고 어떻게 작동하는지 알아보고 기본 사항부터 시...
3,2023-07-26,https://docs.samsungknox.com/admin/knox-captur...,KCAP,문제 해결,문제 해결.앱에서 지원에 액세스하세요. 내에서 지원에 액세스하려면 프로필 목록 레이...
4,2023-10-16,https://docs.samsungknox.com/admin/knox-capture,KCAP,녹스 캡처,녹스 캡처.Knox Capture는 엔터프라이즈 IT 관리자가 조직의 삼성 갤럭시 ...
...,...,...,...,...,...
129,2023-07-26,https://docs.samsungknox.com/admin/knox-config...,KC,Knox Configure를 사용하기에 가장 적합한 회사 유형은 무엇입니까?,Knox Configure를 사용하기에 가장 적합한 회사 유형은 무엇입니까?Knox...
130,2023-07-26,https://docs.samsungknox.com/admin/knox-config...,KC,최종 사용자는 등록 후 Knox Configure에서 삼성 디바이스의 연결을 해제할...,최종 사용자는 등록 후 Knox Configure에서 삼성 디바이스의 연결을 해제할...
131,2023-07-26,https://docs.samsungknox.com/admin/knox-config...,KC,Samsung은 고객을 위해 부팅/종료 애니메이션과 배경 화면을 사용자 정의합니까?,Samsung은 고객을 위해 부팅/종료 애니메이션과 배경 화면을 사용자 정의합니까?...
132,2023-07-26,https://docs.samsungknox.com/admin/knox-config...,KC,Knox Configure를 사용하는 동안 발생할 수 있는 문제나 질문에 대해 삼성...,Knox Configure를 사용하는 동안 발생할 수 있는 문제나 질문에 대해 삼성...


# 5. Craete new korean Json file

In [15]:
def create_json_file(df, file_name):
    df.to_json(file_name, orient='records', force_ascii=False)
    print(f"{file_name} is generated")

cleaned_json_file_path = "data/poc/preprocessed_json/all_processed_data_ko.json"    
create_json_file(df_post, file_name="data/poc/preprocessed_json/all_processed_data_ko.json")


data/poc/preprocessed_json/all_processed_data_ko.json is generated


In [16]:
df_json = pd.read_json(cleaned_json_file_path)
df_json.head(3)

,last_updated,url,project,title,content
0,2023-10-16,https://docs.samsungknox.com/admin/knox-captur...,KCAP,AR 데모 바코드,AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드...
1,2023-07-26,https://docs.samsungknox.com/admin/knox-captur...,KCAP,동영상,동영상.이 섹션에는 Knox Capture와 관련된 제품 및 사용 방법 비디오가 포...
2,2023-07-26,https://docs.samsungknox.com/admin/knox-captur...,KCAP,개요,개요.Knox Capture가 무엇이고 어떻게 작동하는지 알아보고 기본 사항부터 시...
